**Практическая работа #1**

1. hit rate at k

2. money_precision_at_k

3. recall_at_k
4. money_recall_at_k

5. map@k
6. *NDCG@k - для оценок пользователей (1-3) по учебному примеру из лекции

7. Mean Reciprocal Rank
8. *average_precision для ранжирования по примеру с картинки лекции



In [2]:
import pandas as pd
import numpy as np

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дрогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [90]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]
np.random.seed(0)
prices_recommended = np.random.randint(1e3, size=len(recommended_list))

prices_bought = [prices_recommended[i] if (recommended_list[i] in bought_list) else np.random.randint(1e3) for i in range(len(bought_list))] 

def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    flags = np.isin(bought_list, recommended_list)

    hit_rate = (flags.sum() > 0) * 1

    return hit_rate


hit_rate_at_k(recommended_list, bought_list)

1

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  


In [105]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    bought_list = np.array(bought_list) 

    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]

    relevant_revenue = sum(prices_recommended[np.isin(recommended_list, bought_list)])
    recommend_revenue = sum(prices_recommended)

    precision = relevant_revenue / recommend_revenue
    print("recommended:", recommended_list, 
          "\nprices:", prices_recommended,
          "\nbought:", bought_list)
    print("relevant_revenue:", relevant_revenue, 
          "\nrecommended_revenue:", recommend_revenue)

    return precision
np.random.seed(11)

print("money precision:", money_precision_at_k(recommended_list, bought_list, prices_recommended))

recommended: [ 143  156 1134  991   27] 
prices: [684 559 629 192 835] 
bought: [521  32 143 991]
relevant_revenue: 876 
recommended_revenue: 2899
money precision: 0.30217316315971027


### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

In [102]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    flags = np.isin(bought_list, recommended_list)

    recall = flags.sum() / len(bought_list)
 
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):

    prices_bought = np.array(prices_bought)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    print("recommended_list @k:", recommended_list)
    print("prices_recommended @k:", prices_recommended)
    print("bought_list:", bought_list)
    print("prices_bought:", prices_bought)


    # revenue of relevant recommended items 
    relevant_revenue = sum(prices_recommended[np.isin(recommended_list, bought_list)])
    # revenue of relevant items
    total_revenue = sum(prices_bought)
    
    recall = relevant_revenue / total_revenue
    print("relevant_revenue:", relevant_revenue)
    print("total_revenue:", total_revenue)
    
    return recall

In [103]:
money_recall_at_k = money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought)
print("Money Recall@k:", money_recall_at_k)

recommended_list @k: [ 143  156 1134  991   27]
prices_recommended @k: [684 559 629 192 835]
bought_list: [521, 32, 143, 991]
prices_bought: [684 277 754 192]
relevant_revenue: 876
total_revenue: 1907
Money Recall@k: 0.4593602517042475


## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций


In [116]:

def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(0, k-1):
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i+1)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

    
ap_k(recommended_list, bought_list, k=5)


0.75

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров


In [110]:
def map_k(recommended_list, bought_list, k=5):
    
    n_users = len(bought_list)
    recommended_list = np.array(recommended_list)
    sum_ = 0
    for i in range(0, k-1):
        p_k = precision_at_k(recommended_list, bought_list, k=i+1)
        sum_ += p_k
            
    result = sum_ / n_users
    
    return result

print(map_k(recommended_list, bought_list))

0.5833333333333333


### NDCG@k
Normalized discounted cumulative gain

$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = 1$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


<p style='color:red'>(!) Считаем для первых k рекоммендаций   </p> 
<p style='color:red'>(!) - существуют вариации с другими $discount(i)$</p>  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 



$NDCG = \frac{DCG}{ideal DCG}$

$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [152]:
def discount(flag, i):
    if i <= 2:
        return flag / i
    else:
        return flag / np.log2(i)


def dcg(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    if k:
        recommended_list = recommended_list[:k]
    rec_len = len(recommended_list)

    bool_mask = np.isin(recommended_list, bought_list)
    flags = bool_mask * 1

    sum_ = 0
    for i in range(1, rec_len+1):
        sum_ += discount(flags[i-1], i)
    dcg_score = sum_ / rec_len

    return dcg_score


def ideal_dcg(rec_len):
    flags = np.ones(rec_len)
    sum_ = 0
    for i in range(1, rec_len+1):
        sum_ += discount(flags[i-1], i)
    ideal_dcg = sum_ / rec_len

    return ideal_dcg


def ndcg(recommended_list, bought_list, k=5):
    dcg_score = dcg(recommended_list, bought_list, k)

    if k:
        rec_len = k   
    else:
        rec_len = len(recommended_list)
    
    return dcg_score / ideal_dcg(rec_len)


print("DCG Score =", dcg(recommended_list, bought_list, k=None))
print("NDCG Score =", ndcg(recommended_list, bought_list, k=None))
print()
print("DCG Score@5 =", dcg(recommended_list, bought_list))
print("NDCG Score@5 =", ndcg(recommended_list, bought_list))

DCG Score = 0.15
NDCG Score = 0.31549095204264654

DCG Score@5 = 0.3
NDCG Score@5 = 0.489938890671454


### MRR@k
Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [166]:
def mean_reciprocal_rank(recommended_list, bought_list, k=5):
    ranks = []
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    for c in bought_list:
        if c in recommended_list:
            ranks.append(list(recommended_list).index(c)+1)
    
    reciprocal_ranks = list(map(lambda x: x**(-1), ranks))
    mrr = np.mean(reciprocal_ranks)

    return mrr

mrr = mean_reciprocal_rank(recommended_list, bought_list)
print("Mean Reciprocal Rank@5 =", mrr)

Mean Reciprocal Rank@5 = 0.625
